In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
       # Don't wrap wide columns

In [3]:
from psql_models import read_table_async,DiscussionPoint,Meeting,Speaker,Output
from pydantic_models import MeetingSummary
import string
import random

import pandas as pd
# Set display options to show all content
pd.set_option('display.max_colwidth', 100)  # Show full contents of each column
pd.set_option('display.max_rows', 20)      # Show all rows
pd.set_option('display.max_columns', 20)   # Show all columns
pd.set_option('display.width', 100)  

In [4]:

discussion_points_df = await read_table_async(DiscussionPoint)
meetings_df = await read_table_async(Meeting)
speakers_df = await read_table_async(Speaker)

In [5]:
# ... existing code ...

from sqlalchemy import select
from psql_models import async_session, DiscussionPoint, Meeting, Speaker

async def fetch_joined_data():
    async with async_session() as session:
        # Original query remains the same
        query = select(DiscussionPoint, Meeting, Speaker).join(
            Meeting, DiscussionPoint.meeting_id == Meeting.meeting_id
        ).join(
            Speaker, DiscussionPoint.speaker_id == Speaker.id
        )
        
        result = await session.execute(query)
        rows = result.fetchall()
        
        # Create a dictionary to store all speakers per meeting
        meeting_speakers = {}
        for dp, meeting, speaker in rows:
            if meeting.meeting_id not in meeting_speakers:
                meeting_speakers[meeting.meeting_id] = set()
            meeting_speakers[meeting.meeting_id].add(speaker.name)
        
        # Convert the result to a list of dictionaries with other speakers
        data = []
        for dp, meeting, speaker in rows:
            # Get all speakers except the current one
            other_speakers = list(meeting_speakers[meeting.meeting_id] - {speaker.name})
            
            data.append({
                # Original fields remain the same
                'summary_index': dp.summary_index,
                'summary': dp.summary,
                'details': dp.details,
                'referenced_text': dp.referenced_text,
                'topic_name': dp.topic_name,
                'topic_type': dp.topic_type,
                'meeting_id': meeting.meeting_id,
                'meeting_timestamp': meeting.timestamp,
                'speaker_name': speaker.name,
                # Add new field
                'other_speakers': other_speakers
            })

        df = pd.DataFrame(data)
        return df

# Fetch the joined data
joined_df = await fetch_joined_data()
joined_df = joined_df.sort_values(by='meeting_timestamp').reset_index(drop=True)

joined_df['meeting_time'] = pd.to_datetime(joined_df['meeting_timestamp']).dt.strftime('%Y-%m-%d')

In [6]:
len(joined_df)

3194

In [7]:
from vexa import VexaAPI
vexa = VexaAPI()
meetings = await vexa.get_meetings()

Request URL: http://127.0.0.1:8001/api/v1/calls/all
Request Params: {'token': '3ae04e20124d40babc5107e658c666b6'}


In [8]:
meeting_id = meetings[-8]['id']
trasncription = await vexa.get_transcription(meeting_session_id=meeting_id, use_index=False)
if trasncription:   
    df, formatted_input, start_datetime, speakers, transcript = trasncription

In [9]:
meeting_df = joined_df[joined_df['meeting_id'].astype(str)==meeting_id]

In [162]:
r = await MeetingSummary.extract(formatted_input, meeting_df,use_cache=False)

In [163]:
from IPython.display import Markdown
Markdown(r.summary)

Dmitriy Grankin discussed the functionality of a new call assistant tool that records audio and allows users to chat during or after calls, enhancing communication and record-keeping (Call Assistant Tool Functionality)[1]. Sergio Goriachev shared insights from his experience as a solo founder of a startup in Portugal, focusing on the challenges and successes of raising angel investments (Startup Insights from Portugal)[2]. Dmitriy provided an update on his startup VEX, noting a gradual increase in users and the need for further development based on user feedback (VEX User Growth and Feedback)[4]. Sergio proposed integrating AI into travel planning and accommodation services to enhance user experience, suggesting a personalized assistant for finding accommodations (AI Integration in Travel Services)[5]. Both speakers emphasized the importance of understanding customer needs before product development (Understanding Customer Needs)[6]. Sergio explored the concept of tokenizing real estate to make investments more accessible (Tokenization of Real Estate)[7]. Dmitriy mentioned his focus on real-world assets (RWA) and the potential for liquidity in investments (RWA)[3]. They discussed the relevance of AI in their projects (AI)[0] and the importance of robotics for future developments (Robotics)[22]. Sergio described a crypto index being developed by his startup, which collects data based on market caps (Crypto Index)[12]. He also shared his background as a Data Quality Engineer and his experience with big data (Data Quality Engineer)[13][14]. Sergio mentioned CyberFant, a venture capital firm, and their investment in a startup creating personalized assistants (CyberFant)[15]. Dmitriy expressed interest in Alcor's cryonics services and their relevance to his long-term goals (Alcor)[23]. They discussed their experiences living in Argentina, Portugal, Turkey, and other countries, highlighting cultural differences and potential business opportunities (Argentina)[18], (Portugal)[19], (Turkey)[16], (Colombia)[20], (Brazil)[21], (France)[25], (Spain)[24]. Sergio mentioned the unique linguistic adaptations in Argentina, referring to Tokasteshan (Tokasteshan)[17].

In [164]:
meeting_df = meeting_df.set_index('summary_index').sort_index()

In [165]:
from sampling import WeightedSampler

sampler = WeightedSampler(
        df=joined_df,
        text_columns=['summary', 'topic_name', 'details','referenced_text','speaker_name'],
        date_column='meeting_time',
        decay_factor=0.
    )

Building vector index...


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

Index built successfully!


In [191]:
q = 'CyberFant'

In [192]:

sampled_df = sampler.sample(
        query=q,
        n_samples=200,
        recency_weight=0.,
        similarity_weight=1.
    ).sort_values('meeting_timestamp')


In [193]:
from core import generic_call_stream,system_msg,user_msg,assistant_msg


In [194]:
context = sampled_df[['meeting_time','speaker_name','topic_name','summary','details']].to_csv()

In [195]:
from core import count_tokens
count_tokens(context)



12484

In [196]:
r = await generic_call_stream([
    system_msg(f'''{prompts.perplexity}. 
               You user is Dmitry Grankin, who is your user, who also participates the meetings in the context. 
               '''),
    system_msg(f'''Today is {pd.to_datetime('today').strftime('%Y-%m-%d')}.'''),
    system_msg('''Frequently misspelled names and terms: {misspelled}. Replace misspelled company names and terms if found in context.'''),
    
    user_msg(f'search results: {context}. First value of the row is index. Give comprehensive long report.'),
    
    user_msg(f'create comprehencive consice streight to the point report on the topic: {q}, based solely on the context.')
],model='default')

The search results do not contain any information regarding "CyberFant." Therefore, I cannot provide a report on this topic. If you have any other queries or need information on a different subject, please let me know.


In [197]:
from IPython.display import Markdown
Markdown(r)

The search results do not contain any information regarding "CyberFant." Therefore, I cannot provide a report on this topic. If you have any other queries or need information on a different subject, please let me know.

In [ ]:
Markdown(r)

Olga Nemirovskaya discusses several aspects of SEO in the context of their marketing strategies. She emphasizes the importance of internal linking and domain authority as critical factors for SEO success. Olga explains that internal linking helps improve search rankings and that building domain authority through quality backlinks is essential for enhancing visibility.

Additionally, she mentions the need for clear messaging on their website to improve client understanding and engagement, which ties into SEO practices. Olga also highlights the significance of using tools like Surfer SEO for optimizing content based on competitive analysis, indicating that they have been using it successfully for several years to improve their articles' performance in search engines[1][2][3][4].